In [1]:
import pandas as pd
import datetime as dt
from datetime import date
import numpy as np
import re

today = date.today() 
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")

datapath = f"/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/export_tiki_{today}"
filepath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"

In [2]:
df_active = pd.read_csv(filepath +"/active_participants.csv")
df_active.rename(columns = {"Pseudonym ": "customer", "EMA ID": "ema_id", "Ende EMA Baseline": "end_ema", "Status": "status",
                            "Start EMA Baseline": "start_ema", "Studienversion":"study_version", "FOR_ID":"for_id"}, inplace=True)

#df_active = df_active[["customer", "ema_id", "end_ema", "start_ema", "study_version", "for_id", "status"]]
df_active = df_active[["customer","for_id","start_ema", "end_ema", "status"]]
df_active["customer"] = df_active.customer.str[:4]

In [3]:
answers = pd.read_csv(datapath + "/answers.csv")
answers["user"] = answers["user"].str[:4]
answers = answers[["user", "questionnaireSession", "questionnaire", "study", 
                   "question", "order","element", "createdAt"]]
answers["createdAt"] = (pd.to_datetime(answers["createdAt"],unit='ms'))
answers.rename(columns={"user":"customer","questionnaireSession":"session_unique"}, inplace=True)

/Users/leonahammelrath/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
choice = pd.read_csv(datapath + "/choice.csv")
choice = choice[["element", "choice_id", "text", "question"]]
choice.rename(columns={"text":"choice_text"}, inplace=True)

In [5]:
questions = pd.read_csv(datapath + "/questions.csv")
questions = questions[["id", "title"]]
questions.rename(columns={"id":"question","title":"quest_title"}, inplace=True)

In [6]:
questionnaire = pd.read_csv(datapath + "/questionnaires.csv")
questionnaire = questionnaire[["id", "name"]]
questionnaire.rename(columns={"id":"questionnaire","name":"questionnaire_name"}, inplace=True)

In [7]:
answer_merged = pd.merge(answers, choice, on= ["question","element"])
answer_merged = pd.merge(answer_merged, questions, on= "question")
answer_merged = pd.merge(answer_merged, questionnaire, on= "questionnaire")

In [8]:
answer_merged = pd.merge(df_active, answer_merged, on="customer")

In [9]:
answer_merged['weekday'] = answer_merged['createdAt'].dt.day_name()

In [10]:
answer_merged['quest_nr'] = answer_merged['questionnaire_name'].apply(lambda x: int(re.search(r'\d+', x).group()) \
                                               if re.search(r'\d+', x) else None)

In [11]:
answer_merged = answer_merged.loc[answer_merged.study.isin([24,25])]

In [12]:
answer_merged = answer_merged.loc[answer_merged.question.isin([340, 352])]

In [13]:
# Sort by UserID and Date
answer_merged = answer_merged.sort_values(by=['customer', 'createdAt'])

# Calculate the day of assessment
answer_merged['assessment_day'] = answer_merged.groupby('customer')['createdAt'].transform(lambda x: (x - x.min()).dt.days + 1)


In [14]:
answer_merged['assessment_id'] = answer_merged.\
apply(lambda row: f"{row['assessment_day']}_{row['quest_nr']}", axis=1)
